In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import yaml

In [2]:
# Load yaml config file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Load data
sp = pd.read_csv(config["Prices_path"])
sp["Date"] = pd.to_datetime(sp["Date"])
sp.set_index("Date", inplace=True)

In [7]:
def optimize(mean, cov_fixed):
    # Is covariance matrix positive definite? Check if all eigenvalues are positive
    for eig in np.linalg.eigvals(cov_fixed):
        if eig <= 0:
            print("Covariance matrix is not positive definite")
            break

    w = cp.Variable(len(mean))
    objective = cp.Maximize(mean.T @ w)
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=True)

    return w.value

In [8]:
# Generate dates for the beggining of each month
dates = pd.date_range(start=sp.index[0], end=sp.index[-1], freq="MS")
dates

DatetimeIndex(['2010-02-01', '2010-03-01', '2010-04-01', '2010-05-01',
               '2010-06-01', '2010-07-01', '2010-08-01', '2010-09-01',
               '2010-10-01', '2010-11-01',
               ...
               '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01',
               '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01',
               '2024-02-01', '2024-03-01'],
              dtype='datetime64[ns]', length=170, freq='MS')

In [9]:
monhts_past = 3

for date in dates:
    sp_past_window = sp.loc[date - pd.DateOffset(months=monhts_past) : date]
    # drop lines with NaN
    sp_past_window_returns = sp_past_window.pct_change().fillna(0)
    sp_past_window_returns = sp_past_window_returns.iloc[1:] # drop first line with Nan

    # Mean and covariance matrix
    mean = sp_past_window_returns.mean().values
    cov = sp_past_window.cov().values

    cov_fixed = cov + 0.1 * np.identity(len(mean))
    
    w = optimize(mean, cov_fixed)
    print(w)

    break

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 21 11:00:38 PM: Your problem has 561 variables, 2 constraints, and 0 parameters.
(CVXPY) Mar 21 11:00:38 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 21 11:00:38 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 21 11:00:38 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 21 11:00:38 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 21 11:00:38 PM: Compiling problem (target solver=ECOS).
(CV

c:\Users\fotis\Documents\Centrale - 2A\Projet-ST7\env\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
optimize(mean, cov_fixed)

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
Date = pd.to_datetime("2020-01-02")
monhts_past = 3

# Get the stock data for the past months_past months
sp_past_window = sp.loc[Date - pd.DateOffset(months=monhts_past) : Date]
# drop lines with NaN
sp_past_window_returns = sp_past_window.pct_change().fillna(0)
sp_past_window_returns = sp_past_window_returns.iloc[1:] # drop first line with Nan


# Mean and covariance matrix
mean = sp_past_window_returns.mean().values
cov =sp_past_window.cov().values

cov_fixed = cov + 0.1 * np.identity(len(mean))
